In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# validate_shacl.py — Validation SHACL (Brick) d'un graphe Turtle

import sys, argparse
from pathlib import Path
from rdflib import Graph
from pyshacl import validate

# ---------- util ----------
def guess_format(p: Path) -> str:
    suf = p.suffix.lower()
    return {"ttl":"turtle","nt":"nt","nq":"nquads","trig":"trig","jsonld":"json-ld"}.get(suf,"turtle")

def ensure_file(p: Path, name: str) -> Path:
    if not p.exists():
        raise FileNotFoundError(f"{name} introuvable: {p}")
    return p

# ---------- run ----------
def run_validation(data_path: Path, shapes_path: Path, outdir: Path,
                   inference: str = "rdfs", advanced: bool = False,
                   abort_on_first: bool = False):
    data_path  = ensure_file(data_path,  "DATA")
    shapes_path= ensure_file(shapes_path,"SHAPES")
    outdir.mkdir(parents=True, exist_ok=True)

    # Charger les graphes
    g_data   = Graph().parse(str(data_path),   format=guess_format(data_path))
    g_shapes = Graph().parse(str(shapes_path), format=guess_format(shapes_path))

    conforms, report_g, report_text = validate(
        data_graph=g_data,
        shacl_graph=g_shapes,
        inference=inference,              # none | rdfs | owlrl
        advanced=advanced,
        abort_on_first=abort_on_first,
        meta_shacl=False,
        allow_infos=True,
        allow_warnings=True,
    )

    base = data_path.stem
    rep_ttl  = outdir / f"{base}_shacl_report.ttl"
    rep_html = outdir / f"{base}_shacl_report.html"

    report_g.serialize(destination=str(rep_ttl), format="turtle")
    rep_html.write_text(
        f"<html><meta charset='utf-8'><body>"
        f"<h2>Conforms: {conforms}</h2><pre>{report_text}</pre>"
        f"</body></html>", encoding="utf-8"
    )

    print(f"Conforms: {conforms}")
    print(f"Report TTL : {rep_ttl}")
    print(f"Report HTML: {rep_html}")
    return 0 if conforms else 1

# ---------- CLI ----------
def main():
    dflt_shapes = Path("brick_shacl.ttl")  # votre fichier uploadé
    parser = argparse.ArgumentParser(description="Validation SHACL (Brick)")
    parser.add_argument("--data",   required=True, help="Chemin du graphe à valider (TTL/NT/JSON-LD)")
    parser.add_argument("--shapes", default=str(dflt_shapes), help="Shapes SHACL (TTL). Par défaut: brick_shacl.ttl")
    parser.add_argument("--outdir", default="~/DTE/jne_project/validation", help="Dossier sortie")
    parser.add_argument("--inference", choices=["none","rdfs","owlrl"], default="rdfs")
    parser.add_argument("--advanced", action="store_true")
    parser.add_argument("--abort_on_first", action="store_true")
    args = parser.parse_args()

    data   = Path(args.data).expanduser().resolve()
    shapes = Path(args.shapes).expanduser().resolve()
    outdir = Path(args.outdir).expanduser().resolve()

    rc = run_validation(data, shapes, outdir,
                        inference=args.inference,
                        advanced=args.advanced,
                        abort_on_first=args.abort_on_first)
    sys.exit(rc)

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --data DATA [--shapes SHAPES]
                             [--outdir OUTDIR] [--inference {none,rdfs,owlrl}]
                             [--advanced] [--abort_on_first]
ipykernel_launcher.py: error: the following arguments are required: --data


SystemExit: 2